In [8]:
from dagster import op
import pandas as pd
from utils import get_item_info
from bs4 import BeautifulSoup
import requests as r
import time
import numpy as np

FAKE_HISTORY = ['http://google.com',
                'http://hh.ru',
                'https://hh.ru/search/vacancy?area=&fromSearchLine=true&text=',
                'http://avito.ru',
                'https://www.avito.ru/moskva/nedvizhimost']

HEADERS={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}


@op(description='collect raw html data')
def get_pages(context,urls:list)->list:
    pages = []

    with r.Session() as s:

        s.headers.update(HEADERS)
        [s.get(i) for i in FAKE_HISTORY];

        for url in urls:

            time.sleep(np.random.poisson(3))

            response = s.get(url)

            if response.status_code != 200:
                
                raise ConnectionError(f'{response.status_code}\n{response.text}')

            pages.append(response)
            
    return pages

In [9]:
pages = get_pages(None,['https://www.avito.ru/moskva/kvartiry/sdam/na_dlitelnyy_srok-ASgBAgICAkSSA8gQ8AeQUg?cd=1&f=ASgBAQECA0SSA8gQ8AeQUsDBDbr9NwFAzAg0kFmOWYxZAUXoBxV7ImZyb20iOjM1LCJ0byI6bnVsbH0'])

In [83]:
def get_item_info(item):
    item_desc = {}
    try:
        item_desc['pages'] = item.find('div',{'data-marker':'pagination-button'})
        item_desc['datetime'] = pd.to_datetime('now')
        item_desc['publish_delta'] = item.find('div',{'data-marker':'item-date'}).text
        item_desc['id'] = item['id']
        item_desc['url'] = item.a['href']
        item_desc['title'] = item.a['title']
        item_desc['text'] = item.meta['content']
        item_desc['price'] = item.find('meta',{'itemprop':'price'})['content']
        item_desc['fee'] = item.find('span',{'class':'price-noaccent-X6dOy price-textWithoutDiscount-a1bS9 text-text-LurtD text-size-s-BxGpL'}).text
        item_desc['adress'] = item.find('div',{'data-marker':'item-address'}).span.text
        item_desc['metro_dist'] = item.find('span',{'class':'geo-periodSection-bQIE4'}).text
        item_desc['metro'] = item.find('div',{'class':'geo-georeferences-SEtee text-text-LurtD text-size-s-BxGpL'}).text.replace(item_desc['metro_dist'],'')
        item_desc['metro_branch'] = item.find('i',{'class':'geo-icon-Cr9YM'})['style'].replace('background-color:','')
    except:
        pass

    return item_desc

In [84]:

@op(description='extract structure data from html')
def get_item_list(html_pages:list)-> pd.DataFrame: 
    data = pd.DataFrame()
    for html_data in html_pages:
        bs_data = BeautifulSoup(html_data.text,features='lxml')
        item_list = bs_data.find('div',{'class':'items-items-kAJAg'}).findAll('div',{'data-marker':'item'})
        df_scheme = [get_item_info(i) for i in item_list]
        data = data.append(pd.DataFrame(df_scheme))
    return data

In [85]:

@op(description='add features and humanize data')
def featuring_data(data:pd.DataFrame)->pd.DataFrame:
    df = data.copy()
    df['price'] = df['price'].astype(float)
    df['street']=df['adress'].str.extract('(.*?), (?=\d.*)')
    df['n_rooms']=df['title'].str.extract('«(.*?),')
    df['m2'] = df['title'].str.extract(', (\d+).*м²').astype(float)
    df[['floor','max_floor']] = df['title'].str.extract('(\d+/\d+).*эт')[0].str.split('/',expand=True).astype(float)
    df['text'] = df['text'].str.replace('\n','')
    df['rubm2'] = df['price'] / df['m2']
    df.drop(['title','adress','metro'],axis=1,inplace=True)
    return df

In [86]:
zz = get_item_list(pages)

In [88]:
bs_data = BeautifulSoup(pages[0].text,features='lxml')

In [105]:
bs_data.find('div',{'class':'pagination-root-Ntd_O'}).findAll('span',{'class':'pagination-item-JJq_j'})[-2].text

'100'

In [73]:
rr = featuring_data(zz)

In [77]:
rr

,datetime,publish_delta,id,url,text,price,fee,metro_dist,metro_branch,street,n_rooms,m2,floor,max_floor,rubm2
0,2022-07-10 11:08:00.052464,5 минут назад,i2467951328,/moskva/kvartiry/2-k._kvartira_42m_25et._24679...,Вашему вниманию предлагается прекрасная кварти...,50000.0,60% комиссия,11–15 мин.,#0072BA,Партизанская ул.,2-к. квартира,42.0,2.0,5.0,1190.476190
1,2022-07-10 11:08:00.053293,3 минуты назад,i2003686984,/moskva/kvartiry/2-k._kvartira_50m_25et._20036...,Собственник — предлагает сдать на длительный с...,44500.0,NaN,NaN,NaN,NaN,2-к. квартира,50.0,2.0,5.0,890.000000
2,2022-07-10 11:08:00.053791,49 минут назад,i2489952048,/moskva/kvartiry/2-k._kvartira_54m_417et._2489...,"Сдаётся просторная, аккуратная, двухкомнатная ...",48000.0,80% комиссия,11–15 мин.,#E42313,ул. Академика Анохина,2-к. квартира,54.0,4.0,17.0,888.888889
3,2022-07-10 11:08:00.054493,3 часа назад,i2447219921,/moskva/kvartiry/2-k._kvartira_45m_35et._24472...,На длительный срок сдаётся очень уютная двухко...,36000.0,50% комиссия,6–10 мин.,#BED12C,Таганрогская ул.,2-к. квартира,45.0,3.0,5.0,800.000000
4,2022-07-10 11:08:00.055234,23 часа назад,i954350694,/moskva/kvartiry/2-k._kvartira_45m_35et._95435...,Сдается очень светлая и уютная двухкомнатная к...,50000.0,NaN,NaN,NaN,NaN,2-к. квартира,45.0,3.0,5.0,1111.111111
5,2022-07-10 11:08:00.055622,1 час назад,i2220287739,/moskva/kvartiry/1-k._kvartira_35m_25et._22202...,"Чистая светлая, теплая уютная квартира. Сдаю к...",49000.0,30% комиссия,6–10 мин.,#4FB04F,Красноармейская ул.,1-к. квартира,35.0,2.0,5.0,1400.000000
6,2022-07-10 11:08:00.056361,23 часа назад,i2457743625,/moskva/kvartiry/2-k._kvartira_458m_79et._2457...,Рассмотрим всех платеже способных.Сдаётся чист...,29000.0,45% комиссия,16–20 мин.,#FFCD1C,Солнцевский пр-т,2-к. квартира,45.0,7.0,9.0,644.444444
7,2022-07-10 11:08:00.057053,15 часов назад,i2466682398,/moskva/kvartiry/2-k._kvartira_40m_817et._2466...,Сдаётся евродвушка в новом жилом комплексе. Ку...,60000.0,60% комиссия,21–30 мин.,#ADACAC,Олонецкая ул.,2-к. квартира,40.0,8.0,17.0,1500.000000
8,2022-07-10 11:08:00.057791,6 дней назад,i2476363255,/moskva/kvartiry/1-k._kvartira_42m_537et._2476...,Сдается 1-ком кв. 42 метра в «Level Амурская» ...,45000.0,50% комиссия,16–20 мин.,#E42313,Амурская ул.,1-к. квартира,42.0,5.0,37.0,1071.428571
9,2022-07-10 11:08:00.058455,4 дня назад,i2444984783,/moskva/kvartiry/2-k._kvartira_89m_1185et._244...,Стильный угловой апартамент в аренду От Собств...,280000.0,NaN,NaN,NaN,NaN,2-к. квартира,89.0,11.0,85.0,3146.067416
